# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [11]:
import pandas as pd
def cargar_datos():
    # Read the fixed-width file
    datos = pd.read_fwf('datos.dat', 
                        widths=[10, 30, 10, 5], 
                        names=['fecha', 'producto', 'precio', 'cantidad'])
    
    # Convert fecha to datetime, precio and cantidad to numeric
    datos['fecha'] = pd.to_datetime(datos['fecha'])
    datos['precio'] = pd.to_numeric(datos['precio'])
    datos['cantidad'] = pd.to_numeric(datos['cantidad'])
    
    return datos

datos = cargar_datos()
print(datos)

        fecha    producto  precio  cantidad
0  2024-08-27     Mirinda    1510        14
1  2024-08-27     Mirinda    1560        12
2  2024-08-28     Torasso     940         8
3  2024-08-29  Pepsi Cola    1210        10
4  2024-08-30     Mirinda    1520         1
5  2024-09-01     Mirinda    1550        15
6  2024-09-01      Sprite     810         4
7  2024-09-02   Coca Cola    1100         4
8  2024-09-02  Pepsi Cola    1220        13
9  2024-09-02     Torasso     910         5
10 2024-09-02     Torasso     920         3
11 2024-09-03   Coca Cola    1020         8
12 2024-09-03     Mirinda    1570         7
13 2024-09-03     Mirinda    1590         2
14 2024-09-04  Pepsi Cola    1220        13
15 2024-09-05     Mirinda    1500         3
16 2024-09-05  Pepsi Cola    1300         5
17 2024-09-06   Coca Cola    1080         1
18 2024-09-06      Sprite     860        12
19 2024-09-06     Torasso     930         3
20 2024-09-07   Coca Cola    1080        14
21 2024-09-07      Sprite     87

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [12]:
def calcular_totales(datos):
    total_importe = (datos['precio'] * datos['cantidad']).sum()
    total_cantidad = datos['cantidad'].sum()
    return total_importe, total_cantidad

importe, cantidad = calcular_totales(datos)
print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")

Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [13]:
def unidades_vendidas(datos):
    return datos.groupby('producto')['cantidad'].sum()

ventas = unidades_vendidas(datos)
print(ventas)

producto
Coca Cola     57
Mirinda       85
Pepsi Cola    89
Sprite        72
Torasso       32
Name: cantidad, dtype: int64


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [14]:
def precio_promedio(datos):
    return datos.groupby('producto')['precio'].mean()

precios = precio_promedio(datos)
print(precios)

producto
Coca Cola     1072.500000
Mirinda       1545.833333
Pepsi Cola    1245.000000
Sprite         841.428571
Torasso        920.000000
Name: precio, dtype: float64


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [15]:
def ranking_productos(datos, top=3):
    return datos.groupby('producto')['cantidad'].sum().sort_values(ascending=False).head(top)

ranking = ranking_productos(datos)
print(ranking)

producto
Pepsi Cola    89
Mirinda       85
Sprite        72
Name: cantidad, dtype: int64


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [16]:
def ventas_por_mes(datos):
    datos['mes'] = datos['fecha'].dt.to_period('M')
    return datos.groupby(['mes', 'producto'])['cantidad'].sum().unstack(fill_value=0)

ventas_mensuales = ventas_por_mes(datos)
print(ventas_mensuales)

producto  Coca Cola  Mirinda  Pepsi Cola  Sprite  Torasso
mes                                                      
2024-08           0       27          10       0        8
2024-09          57       58          79      72       24


### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [17]:
def resumen_ventas(datos):
    resumen = datos.groupby('producto').agg({
        'precio': 'mean',
        'cantidad': 'sum',
        'precio': lambda x: (x * datos.loc[x.index, 'cantidad']).sum()
    }).rename(columns={'precio': 'importe_total', 'cantidad': 'unidades_vendidas'})
    resumen['precio_promedio'] = resumen['importe_total'] / resumen['unidades_vendidas']
    return resumen.sort_index()

resumen = resumen_ventas(datos)
print(resumen)

            importe_total  unidades_vendidas  precio_promedio
producto                                                     
Coca Cola           60780                 57      1066.315789
Mirinda            131080                 85      1542.117647
Pepsi Cola         110510                 89      1241.685393
Sprite              61040                 72       847.777778
Torasso             29320                 32       916.250000


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [18]:
import numpy as np


def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return y - y_pred

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

def buscar_coeficientes():
    for a in range(-10, 11):
        for b in range(-10, 11):
            for c in range(-10, 11):
                coeficientes = np.array([a, b, c])
                y_pred = f(X, coeficientes)
                if np.all(np.abs(error(Y, y_pred)) < 1e-10):
                    return coeficientes
    return None

coeficientes = buscar_coeficientes()
print("Ejercicio 8:")
print(f"Los coeficientes son {coeficientes}")

Ejercicio 8:
Los coeficientes son [-2 10  0]


### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [19]:
X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

def buscar_coeficientes():
    coeficientes = np.random.randint(-10, 11, 3)
    mejor_error = float('inf')
    aprendizaje = 0.001
    
    while mejor_error > 1:
        y_pred = f(X, coeficientes)
        error_actual = error(Y, y_pred)
        error_total = np.sum(error_actual**2)
        
        if error_total < mejor_error:
            mejor_error = error_total
        
        nuevos_coeficientes = coeficientes + np.random.uniform(-aprendizaje, aprendizaje, 3)
        nuevos_y_pred = f(X, nuevos_coeficientes)
        nuevo_error_total = np.sum(error(Y, nuevos_y_pred)**2)
        
        if nuevo_error_total < error_total:
            coeficientes = nuevos_coeficientes
    
    return coeficientes

coeficientes = buscar_coeficientes()
y_pred = f(X, coeficientes)
error_final = np.sum(error(Y, y_pred)**2)
print("\nEjercicio 9:")
print(f"Los coeficientes son {coeficientes} y el error es {error_final}")


Ejercicio 9:
Los coeficientes son [-1.90114185  9.69255709 -0.14532469] y el error es 0.9997901452616865
